## Modeling Stock Movement with Two Signals

You’re working at an **HFT (High-Frequency Trading)** firm and want to model whether a stock (say **AAPL**) will go **up** or **down** tomorrow, based on two signals:

- $S_1$: **Momentum signal** ($1 =$ positive, $0 =$ negative)  
- $S_2$: **News sentiment** ($1 =$ positive, $0 =$ negative)

The **target variable**:

- $Y$: **Stock move tomorrow** ($1 =$ Up, $0 =$ Down)

You’ve collected **historical data for 10 days**:


| Day | S₁ (Momentum) | S₂ (News) | Y (Up/Down) |
| --- | ------------- | --------- | ----------- |
| 1   | 1             | 1         | 1           |
| 2   | 1             | 0         | 1           |
| 3   | 0             | 1         | 0           |
| 4   | 0             | 0         | 0           |
| 5   | 1             | 1         | 1           |
| 6   | 1             | 1         | 1           |
| 7   | 0             | 1         | 0           |
| 8   | 1             | 0         | 1           |
| 9   | 0             | 0         | 0           |
| 10  | 0             | 1         | 0           |


## Goal

You want to:

1. **Model**  
   $$
   P(Y \mid S_1, S_2)
   $$
   using **Bayesian reasoning**

2. **Estimate the parameters** using both **MLE** (Maximum Likelihood Estimation) and **MAP** (Maximum A Posteriori)

3. **Draw a quant-intuitive conclusion** for **trading**



## Step 1: Define the Probabilistic Model

We’ll assume **Naive Bayes independence** between $S_1, S_2$ given $Y$:

$$
P(Y, S_1, S_2) = P(Y) \times P(S_1 \mid Y) \times P(S_2 \mid Y)
$$



---

## Step 2: Compute Joint and Marginal Probabilities

From data:

- Total = 10 samples  
- Count of $Y=1$ (Up) = 5  
- Count of $Y=0$ (Down) = 5  

So:

$$
P(Y=1) = 0.5, \quad P(Y=0) = 0.5
$$

**Conditional Probabilities (empirical):**

For $Y=1$:

$$
P(S_1=1 \mid Y=1) = 1, \quad P(S_2=1 \mid Y=1) = 0.6
$$

For $Y=0$:

$$
P(S_1=1 \mid Y=0) = 0, \quad P(S_2=1 \mid Y=0) = 0.6
$$



---

## Step 3: Likelihood — MLE Estimation

MLE simply takes empirical frequencies as parameters.  
So your estimated model is:

$$
P(Y=1) = 0.5
$$

$$
P(S_1=1 \mid Y=1) = 1, \quad P(S_1=1 \mid Y=0) = 0
$$

$$
P(S_2=1 \mid Y=1) = 0.6, \quad P(S_2=1 \mid Y=0) = 0.6
$$


---

## Step 4: Prediction (Using Joint → Conditional)

Now suppose tomorrow you observe:

$$
S_1 = 1, \quad S_2 = 1
$$

We want to compute:

$$
P(Y=1 \mid S_1=1, S_2=1)
$$

By Bayes’ rule:

$$
P(Y=1 \mid S_1, S_2) = 
\frac{P(S_1, S_2 \mid Y=1) \, P(Y=1)}{P(S_1, S_2)}
$$

Ignoring the denominator (we’ll normalize later), compute numerator for both $Y$ values:

$$
\text{Score}(Y=1) = P(S_1=1 \mid Y=1) \times P(S_2=1 \mid Y=1) \times P(Y=1)
= 1 \times 0.6 \times 0.5 = 0.3
$$

$$
\text{Score}(Y=0) = P(S_1=1 \mid Y=0) \times P(S_2=1 \mid Y=0) \times P(Y=0)
= 0 \times 0.6 \times 0.5 = 0
$$

Normalize:

$$
P(Y=1 \mid S_1=1, S_2=1) = \frac{0.3}{0.3 + 0} = 1
$$

**MLE predicts 100% probability** of the stock going up.  
But this is **overconfident**, since it’s learned zero probability for $P(S_1=1 \mid Y=0)$.




---

## Step 5: MAP Estimation (Introduce Prior)

To fix overconfidence, let’s add a **Beta(1,1)** prior (uniform).

For conditional probabilities like $P(S_1=1 \mid Y=0)$, we apply:

$$
\hat{\theta}_{MAP} = \frac{\alpha + k - 1}{\alpha + \beta + n - 2}
$$

For $S_1 \mid Y=0$:

- $\alpha = 1$, $\beta = 1$, $k = 0$ (count of successes), $n = 5$

$$
P(S_1=1 \mid Y=0) = \frac{1 + 0 - 1}{1 + 1 + 5 - 2} = \frac{0}{5} = 0
$$

Still 0 — so let’s use a stronger smoothing prior, say **Beta(2,2)**:

$$
P(S_1=1 \mid Y=0) = \frac{2 + 0 - 1}{2 + 2 + 5 - 2} = \frac{1}{7} = 0.14
$$

Now we avoid total zero.

---

### Recalculate Posterior

$$
\text{Score}(Y=1) = 1 \times 0.6 \times 0.5 = 0.3
$$

$$
\text{Score}(Y=0) = 0.14 \times 0.6 \times 0.5 = 0.042
$$

Normalize:

$$
P(Y=1 \mid S_1=1, S_2=1) = \frac{0.3}{0.3 + 0.042} = 0.877
$$

**MAP predicts 87.7% chance** of the stock going up — more realistic.


## Final Conclusion

- MLE → 100% confident “BUY”

- MAP → 87.7% “BUY”

- A quant strategist prefers MAP because it respects uncertainty and smooths noise.

- This is exactly how Bayesian models reduce overfitting in live trading by combining:

    - Market data (likelihood) + Prior belief (historical experience).